Imports

In [ ]:
import os
import numpy as np
from PIL import Image

### Helper functions

Function to add RGB components

In [ ]:
def AddRGB(rgb1, rgb2):
    rgb_sum = []
    rgb_sum.append(rgb1[0] + rgb2[0])
    rgb_sum.append(rgb1[1] + rgb2[1])
    rgb_sum.append(rgb1[2] + rgb2[2])
    return rgb_sum

Function to subtract RGB components

In [ ]:
def SubRGB(rgb1, rgb2):
    rgb_diff = []
    rgb_diff.append(rgb1[0] - rgb2[0])
    rgb_diff.append(rgb1[1] - rgb2[1])
    rgb_diff.append(rgb1[2] - rgb2[2])
    return rgb_diff

Function to normalize 3d array

In [ ]:
def GetMaxValue(image_array):
    x, y, *_ = image_array.shape
    max_value = 0
    for i in range(x):
        for j in range(y):
            r, g, b = image_array[i][j]
            if r > max_value:
                max_value = r
            elif g > max_value:
                max_value = g
            elif b > max_value:
                max_value = b
    return max_value

def NormalizeArray(image_array):
    x, y, *_ = image_array.shape
    max_value = GetMaxValue(image_array)
    normalized_image_array = np.copy(image_array)
    for i in range(x):
        for j in range(y):
            r, g, b = image_array[i][j]
            r1 = round(r / max_value * 255)
            g1 = round(g / max_value * 255)
            b1 = round(b / max_value * 255)
            normalized_image_array[i][j] = [r1, g1, b1]
    return normalized_image_array

Function to convert 3d array to an image

In [ ]:
def ArrayToImage(image_array):
    x, y, *_ = image_array.shape
    image = Image.new("RGB", (x, y))
    for i in range(x):
        for j in range(y):
            image.putpixel((i, j), tuple(image_array[i][j]))
    return image

### Main functions

CalculateIntegral function

In [ ]:
def CalculateIntegral(image):
    x, y = image.size
    row_sums = np.zeros((x+1, y+1, 3), dtype=np.uint32)
    integral_image = np.zeros((x+1, y+1, 3), dtype=np.uint32)
    for i in range(x):
        for j in range(y):
            r = i+1
            c = j+1
            row_sums[r][c] = AddRGB(row_sums[r][c-1], image.getpixel((i, j)))
            integral_image[r][c] = AddRGB(integral_image[r-1][c], row_sums[r][c])
    integral_image = np.delete(integral_image, 0, axis=0)
    integral_image = np.delete(integral_image, 0, axis=1)
    return integral_image

CalculateLocalSum function

In [ ]:
def CalculateLocalSum(integral_image, p0, p1):
    x0, y0 = p0
    x1, y1 = p1
    sum_x = integral_image[x0][y0]
    sum_y = integral_image[x0][y1]
    sum_z = integral_image[x1][y0]
    sum_k = integral_image[x1][y1]
    local_sum = AddRGB(sum_k, sum_x)
    local_sum = SubRGB(local_sum, sum_z)
    local_sum = SubRGB(local_sum, sum_y)
    return local_sum

### Test

Test 1

In [ ]:
image_array = np.full((200, 200, 3), 100)
image = ArrayToImage(image_array)

In [ ]:
integral_image = CalculateIntegral(image)
ArrayToImage(NormalizeArray(integral_image))

In [ ]:
CalculateLocalSum(integral_image, (20, 20), (30, 30))

Test 2

In [ ]:
current_directory = os.getcwd()
image_path = current_directory + R"\images\f2.png"
image = Image.open(image_path)